# Warping RAG2 mice to MWT brain template

Using the inverse warps to compute Jacobians (manually for different folders)

In [ ]:
%%bash
module load afni/20.1.11

input=/data/bioprotean/RAG2/AVG/MWT_avg/MKO/5_anmwp/5_shft_WARPINV.nii.gz
output=/data/bioprotean/RAG2/AVG/MWT_avg/JCB/MKO_invjcb5.nii

3dNwarpFuncs -nwarp $input -bulk -prefix $output

In [ ]:
%%bash
module load afni/20.1.11

input=/data/bioprotean/RAG2/AVG/MWT_avg/MWT/5_anmwp/5_shft_WARPINV.nii.gz
output=/data/bioprotean/RAG2/AVG/MWT_avg/JCB/MWT_invjcb5.nii

3dNwarpFuncs -nwarp $input -bulk -prefix $output

Computing the mean Jacobian for knockouts

In [ ]:
%%bash
module load afni/20.1.11
input1=/data/bioprotean/RAG2/AVG/MWT_avg/JCB/MKO_invjcb1.nii
input2=/data/bioprotean/RAG2/AVG/MWT_avg/JCB/MKO_invjcb2.nii
input3=/data/bioprotean/RAG2/AVG/MWT_avg/JCB/MKO_invjcb3.nii
input4=/data/bioprotean/RAG2/AVG/MWT_avg/JCB/MKO_invjcb4.nii
input5=/data/bioprotean/RAG2/AVG/MWT_avg/JCB/MKO_invjcb5.nii
output=/data/bioprotean/RAG2/AVG/MWT_avg/JCB/MKO_MWTavg_invjcb_avg.nii
3dMean -prefix $output $input1 $input2 $input3 $input4 $input5

Doing 1-set and 2-set t-tests

In [ ]:
%%bash
module load afni/20.1.11
input1=/data/bioprotean/RAG2/AVG/MWT_avg/JCB/MKO_invjcb1.nii
input2=/data/bioprotean/RAG2/AVG/MWT_avg/JCB/MKO_invjcb2.nii
input3=/data/bioprotean/RAG2/AVG/MWT_avg/JCB/MKO_invjcb3.nii
input4=/data/bioprotean/RAG2/AVG/MWT_avg/JCB/MKO_invjcb4.nii
input5=/data/bioprotean/RAG2/AVG/MWT_avg/JCB/MKO_invjcb5.nii
output=/data/bioprotean/RAG2/AVG/MWT_avg/JCB/MKO_MWTavg_1set_ttest.nii
3dttest++ -prefix $output -setA $input1 $input2 $input3 $input4 $input5

In [ ]:
%%bash
module load afni/20.1.11
cd /data/bioprotean/RAG2/AVG/MWT_avg/JCB/

input1=MKO_invjcb1.nii
input2=MKO_invjcb2.nii
input3=MKO_invjcb3.nii
input4=MKO_invjcb4.nii
input5=MKO_invjcb5.nii

input6=MWT_invjcb1.nii
input7=MWT_invjcb2.nii
input8=MWT_invjcb3.nii
input9=MWT_invjcb4.nii
input10=MWT_invjcb5.nii

3dttest++ \
    -prefix MKO_MWTavg_2set_ttest.nii \
    -setA $input1 $input2 $input3 $input4 $input5 \
    -setB $input6 $input7 $input8 $input9 $input10

**Thresholding the average Jacobian by t-test in the MWT average brain space**
<br>To threshold the average Jacobian by the t-test giving p-value of 0.05:
<br>Brick number 1 is setA-setB_Tstat (Bricks start from 0)
<br>Seen in AFNI that a p-value of 0.05 corresponds to a t-stat (for 2set) of 2.3060
<br>Seen in AFNI that a p-value of 0.05 corresponds to a t-stat (for 1set) of 2.7764

In [ ]:
%%bash
module load afni/20.1.11
cd /data/bioprotean/RAG2/AVG/MWT_avg/Jacobian/threshold/
3dcalc \
    -a MKO_MWTavg_invjcb_avg.nii \
    -b MKO_MWTavg_2set_ttest.nii[1] \
    -expr 'a*astep(b,2.3060)' \
    -datum float \
    -prefix MKO_MWTavg_invjcb_avg_thrsh_2set.nii

In [ ]:
%%bash
module load afni/20.1.11
cd /data/bioprotean/RAG2/AVG/MWT_avg/Jacobian/threshold/
3dcalc \
    -a MKO_MWTavg_invjcb_avg.nii \
    -b MKO_MWTavg_1set_ttest.nii[1] \
    -expr 'a*astep(b,2.7764)' \
    -datum float \
    -prefix MKO_MWTavg_invjcb_avg_thrsh_1set.nii

Warping the MWT average brain to Allen brain template

In [ ]:
%%bash
module load afni/20.1.11
cd /data/bioprotean/RAG2/AVG/MWT_avg/to_allen/
3dAllineate \
    -base allen_brain75.nii \
    -source MWT_template_rs.nii \
    -1Dmatrix_save 1DM \
    -cmass -cost lpa -mi -twopass \
    -automask -num_rtb 2000 -warp affine_general \
    -prefix MWT_template_rs_aff.nii

auto_warp.py \
    -base allen_brain75.nii \
    -input MWT_template_rs_aff.nii \
    -skip_affine yes \
    -qworkhard 0 2 \
    -qw_opts -maxlev 09 -iwarp \
    -output_dir awp

Applying the warp to the windowed average Jacobian and average Jacobian

In [ ]:
%%bash
module load afni/20.1.11
cd /data/bioprotean/RAG2/AVG/MWT_avg/to_allen/02_apply/
3dAllineate \
    -master allen_brain75.nii \
    -source MKO_MWTavg_invjcb_avg.nii \
    -1Dmatrix_apply 1DM.aff12.1D \
    -cmass \
    -prefix MKO_MWTavg_invjcb_avg_aff.nii

In [2]:
%%bash
module load afni/20.1.11
cd /data/bioprotean/RAG2/AVG/MWT_avg/to_allen/02_apply/
3dNwarpApply \
    -source MKO_MWTavg_invjcb_avg_aff.nii \
    -nwarp anat.un.qw_WARP.nii \
    -master allen_brain75.nii \
    -prefix MKO_MWTavg_invjcb_avg_qw.nii

3dNwarpApply: /packages/7x/anaconda3/5.3.0/lib/libuuid.so.1: no version information available (required by /lib64/libSM.so.6)
++ 3dNwarpApply: AFNI version=AFNI_20.1.11 (May 28 2020) [64-bit]
++ Authored by: Zhark the Warped
++ opened source dataset 'MKO_MWTavg_invjcb_avg_aff.nii'
++ -master dataset is 'allen_brain75.nii'
++ Processing -nwarp 
++ Warping:.Z
++ Output dataset ./MKO_MWTavg_invjcb_avg_qw.nii
++ total Elapsed time = 5.4 sec


In [ ]:
%%bash
module load afni/20.1.11
cd /data/bioprotean/RAG2/AVG/MWT_avg/to_allen/apply/
3dAllineate \
    -master allen_brain75.nii \
    -source MKO_MWTavg_invjcb_avg_thrsh_1set.nii \
    -1Dmatrix_apply 1DM.aff12.1D \
    -cmass \
    -prefix MKO_MWTavg_invjcb_avg_thrsh_1set_aff.nii

In [ ]:
%%bash
module load afni/20.1.11
cd /data/bioprotean/RAG2/AVG/MWT_avg/to_allen/apply/
3dNwarpApply \
    -source MKO_MWTavg_invjcb_avg_thrsh_1set_aff.nii \
    -nwarp anat.un.qw_WARP.nii \
    -master allen_brain75.nii \
    -prefix MKO_MWTavg_invjcb_avg_thrsh_1set_qw.nii

In [ ]:
%%bash
module load afni/20.1.11
cd /data/bioprotean/RAG2/AVG/MWT_avg/to_allen/apply/
3dAllineate \
    -master allen_brain75.nii \
    -source MKO_MWTavg_invjcb_avg_thrsh_2set.nii \
    -1Dmatrix_apply 1DM.aff12.1D \
    -cmass \
    -prefix MKO_MWTavg_invjcb_avg_thrsh_2set_aff.nii

In [ ]:
%%bash
module load afni/20.1.11
cd /data/bioprotean/RAG2/AVG/MWT_avg/to_allen/apply/
3dNwarpApply \
    -source MKO_MWTavg_invjcb_avg_thrsh_2set_aff.nii \
    -nwarp anat.un.qw_WARP.nii \
    -master allen_brain75.nii \
    -prefix MKO_MWTavg_invjcb_avg_thrsh_2set_qw.nii

Masking the Jacobians by Allen annotation atlas

In [ ]:
%%bash
module load afni/20.1.11
cd /data/bioprotean/RAG2/AVG/MWT_avg/to_allen/03_mask/
3dcalc \
    -a MKO_MWTavg_invjcb_avg_qw.nii \
    -b allen_annot75.nii \
    -expr 'a*bool(b)' \
    -datum float \
    -prefix MKO_MWTavg_invjcb_avg_qw_masked.nii

In [ ]:
%%bash
module load afni/20.1.11
cd /data/bioprotean/RAG2/AVG/MWT_avg/to_allen/mask/
3dcalc \
    -a MKO_MWTavg_invjcb_avg_thrsh_2set_qw.nii \
    -b allen_annot75.nii \
    -expr 'a*bool(b)' \
    -datum float \
    -prefix MKO_MWTavg_invjcb_avg_thrsh_2set_qw_masked.nii

In [ ]:
%%bash
module load afni/20.1.11
cd /data/bioprotean/RAG2/AVG/MWT_avg/to_allen/mask/
3dcalc \
    -a MKO_MWTavg_invjcb_avg_thrsh_1set_qw.nii \
    -b allen_annot75.nii \
    -expr 'a*bool(b)' \
    -datum float \
    -prefix MKO_MWTavg_invjcb_avg_thrsh_1set_qw_masked.nii

Separating the Jacobians into positive and negative files

In [ ]:
%%bash
module load afni/20.1.11
cd /data/bioprotean/RAG2/AVG/MWT_avg/to_allen/separate/
3dcalc \
    -a MKO_MWTavg_invjcb_avg_thrsh_1set_qw_masked.nii \
    -expr 'posval(a)' \
    -datum float \
    -prefix MKO_MWTavg_invjcb_avg_thrsh_1set_qw_masked_pos.nii

In [ ]:
%%bash
module load afni/20.1.11
cd /data/bioprotean/RAG2/AVG/MWT_avg/to_allen/separate/
3dcalc \
    -a MKO_MWTavg_invjcb_avg_thrsh_2set_qw_masked.nii \
    -expr 'posval(a)' \
    -datum float \
    -prefix MKO_MWTavg_invjcb_avg_thrsh_2set_qw_masked_pos.nii

In [ ]:
%%bash
module load afni/20.1.11
cd /data/bioprotean/RAG2/AVG/MWT_avg/to_allen/separate/
3dcalc \
    -a MKO_MWTavg_invjcb_avg_thrsh_1set_qw_masked.nii \
    -expr '-posval(-a)' \
    -datum float \
    -prefix MKO_MWTavg_invjcb_avg_thrsh_1set_qw_masked_neg.nii

In [ ]:
%%bash
module load afni/20.1.11
cd /data/bioprotean/RAG2/AVG/MWT_avg/to_allen/separate/
3dcalc \
    -a MKO_MWTavg_invjcb_avg_thrsh_2set_qw_masked.nii \
    -expr '-posval(-a)' \
    -datum float \
    -prefix MKO_MWTavg_invjcb_avg_thrsh_2set_qw_masked_neg.nii

Filtering the Jacobian by contrast to get rid of warp effects
<br>As seen in AFNI, threshold of 0.25 for positive ones and 0.4 for negatives chosen

In [ ]:
%%bash
module load afni/20.1.11
cd /data/bioprotean/RAG2/AVG/MWT_avg/to_allen/contrast/
3dcalc \
    -a MKO_MWTavg_invjcb_avg_thrsh_1set_qw_masked_pos.nii \
    -expr 'posval(a-0.25)' \
    -datum float \
    -prefix MKO_MWTavg_invjcb_avg_thrsh_1set_qw_masked_pos_ctrst.nii

In [ ]:
%%bash
module load afni/20.1.11
cd /data/bioprotean/RAG2/AVG/MWT_avg/to_allen/contrast/
3dcalc \
    -a MKO_MWTavg_invjcb_avg_thrsh_2set_qw_masked_pos.nii \
    -expr 'posval(a-0.25)' \
    -datum float \
    -prefix MKO_MWTavg_invjcb_avg_thrsh_2set_qw_masked_pos_ctrst.nii

In [ ]:
%%bash
module load afni/20.1.11
cd /data/bioprotean/RAG2/AVG/MWT_avg/to_allen/contrast/
3dcalc \
    -a MKO_MWTavg_invjcb_avg_thrsh_1set_qw_masked_neg.nii \
    -expr '-posval(-a-0.4)' \
    -datum float \
    -prefix MKO_MWTavg_invjcb_avg_thrsh_1set_qw_masked_neg_ctrst.nii

In [ ]:
%%bash
module load afni/20.1.11
cd /data/bioprotean/RAG2/AVG/MWT_avg/to_allen/contrast/
3dcalc \
    -a MKO_MWTavg_invjcb_avg_thrsh_2set_qw_masked_neg.nii \
    -expr '-posval(-a-0.4)' \
    -datum float \
    -prefix MKO_MWTavg_invjcb_avg_thrsh_2set_qw_masked_neg_ctrst.nii

Using AFNI clusterize function to reomve voxels of noise

In [ ]:
%%bash
module load afni/20.1.11
cd /data/bioprotean/RAG2/AVG/MWT_avg/to_allen/clusterize/
3dclust \
    -NN2 100 \
    -prefix MKO_MWTavg_invjcb_avg_thrsh_2set_qw_masked_neg_ctrst_clust.nii \
    MKO_MWTavg_invjcb_avg_thrsh_2set_qw_masked_neg_ctrst.nii \

In [ ]:
%%bash
module load afni/20.1.11
cd /data/bioprotean/RAG2/AVG/MWT_avg/to_allen/clusterize/
3dclust \
    -NN2 100 \
    -prefix MKO_MWTavg_invjcb_avg_thrsh_2set_qw_masked_pos_ctrst_clust.nii \
    MKO_MWTavg_invjcb_avg_thrsh_2set_qw_masked_pos_ctrst.nii \

In [ ]:
%%bash
module load afni/20.1.11
cd /data/bioprotean/RAG2/AVG/MWT_avg/to_allen/clusterize/
3dclust \
    -NN2 100 \
    -prefix MKO_MWTavg_invjcb_avg_thrsh_1set_qw_masked_neg_ctrst_clust.nii \
    MKO_MWTavg_invjcb_avg_thrsh_1set_qw_masked_neg_ctrst.nii \

In [ ]:
%%bash
module load afni/20.1.11
cd /data/bioprotean/RAG2/AVG/MWT_avg/to_allen/clusterize/
3dclust \
    -NN2 100 \
    -prefix MKO_MWTavg_invjcb_avg_thrsh_1set_qw_masked_pos_ctrst_clust.nii \
    MKO_MWTavg_invjcb_avg_thrsh_1set_qw_masked_pos_ctrst.nii \

Making binary version of the clusterized Jacobians

In [ ]:
%%bash
module load afni/20.1.11
cd /data/bioprotean/RAG2/AVG/MWT_avg/to_allen/binary/
3dcalc \
    -a MKO_MWTavg_invjcb_avg_thrsh_1set_qw_masked_pos_ctrst_clust.nii \
    -expr 'step(a)' \
    -datum float \
    -prefix MKO_MWTavg_invjcb_avg_thrsh_1set_qw_masked_pos_ctrst_clust_binary.nii

In [ ]:
%%bash
module load afni/20.1.11
cd /data/bioprotean/RAG2/AVG/MWT_avg/to_allen/binary/
3dcalc \
    -a MKO_MWTavg_invjcb_avg_thrsh_2set_qw_masked_pos_ctrst_clust.nii \
    -expr 'step(a)' \
    -datum float \
    -prefix MKO_MWTavg_invjcb_avg_thrsh_2set_qw_masked_pos_ctrst_clust_binary.nii

In [ ]:
%%bash
module load afni/20.1.11
cd /data/bioprotean/RAG2/AVG/MWT_avg/to_allen/binary/
3dcalc \
    -a MKO_MWTavg_invjcb_avg_thrsh_1set_qw_masked_neg_ctrst_clust.nii \
    -expr 'step(-a)' \
    -datum float \
    -prefix MKO_MWTavg_invjcb_avg_thrsh_1set_qw_masked_neg_ctrst_clust_binary.nii

In [ ]:
%%bash
module load afni/20.1.11
cd /data/bioprotean/RAG2/AVG/MWT_avg/to_allen/binary/
3dcalc \
    -a MKO_MWTavg_invjcb_avg_thrsh_2set_qw_masked_neg_ctrst_clust.nii \
    -expr 'step(-a)' \
    -datum float \
    -prefix MKO_MWTavg_invjcb_avg_thrsh_2set_qw_masked_neg_ctrst_clust_binary.nii

Resampling to 100 micrometer

In [ ]:
%%bash
module load afni/20.1.11
cd /data/bioprotean/RAG2/AVG/MWT_avg/to_allen/overlap/resample_100/
3dresample \
    -master allen_brain100.nii \
    -input MKO_MWTavg_invjcb_avg_thrsh_1set_qw_masked_neg_ctrst_clust.nii \
    -prefix MKO_MWTavg_invjcb_avg_thrsh_1set_qw_masked_neg_ctrst_clust_rs100.nii

In [ ]:
%%bash
module load afni/20.1.11
cd /data/bioprotean/RAG2/AVG/MWT_avg/to_allen/overlap/resample_100/
3dresample \
    -master allen_brain100.nii \
    -input MKO_MWTavg_invjcb_avg_thrsh_1set_qw_masked_neg_ctrst_clust_binary.nii \
    -prefix MKO_MWTavg_invjcb_avg_thrsh_1set_qw_masked_neg_ctrst_clust_binary_rs100.nii

Resampling to higher resolutions to test visualizations

In [2]:
%%bash
module load afni/20.1.11
cd /data/bioprotean/RAG2/AVG/MWT_avg/to_allen/overlap/high_res/
3dresample \
    -dxyz 0.025 0.025 0.025 \
    -input MKO_MWTavg_invjcb_avg_thrsh_1set_qw_masked_neg_ctrst_clust_75um.nii \
    -prefix MKO_MWTavg_invjcb_avg_thrsh_1set_qw_masked_neg_ctrst_clust_25um.nii

3dresample: /packages/7x/anaconda3/5.3.0/lib/libuuid.so.1: no version information available (required by /lib64/libSM.so.6)


In [3]:
%%bash
module load afni/20.1.11
cd /data/bioprotean/RAG2/AVG/MWT_avg/to_allen/overlap/high_res/
3dresample \
    -dxyz 0.025 0.025 0.025 \
    -input MKO_MWTavg_invjcb_avg_thrsh_1set_qw_masked_neg_ctrst_clust_binary_75um.nii \
    -prefix MKO_MWTavg_invjcb_avg_thrsh_1set_qw_masked_neg_ctrst_clust_binary_25um.nii

3dresample: /packages/7x/anaconda3/5.3.0/lib/libuuid.so.1: no version information available (required by /lib64/libSM.so.6)


In [4]:
%%bash
module load afni/20.1.11
cd /data/bioprotean/RAG2/AVG/MWT_avg/to_allen/overlap/high_res/
3dresample \
    -dxyz 0.015 0.015 0.015 \
    -input MKO_MWTavg_invjcb_avg_thrsh_1set_qw_masked_neg_ctrst_clust_75um.nii \
    -prefix MKO_MWTavg_invjcb_avg_thrsh_1set_qw_masked_neg_ctrst_clust_15um.nii

3dresample: /packages/7x/anaconda3/5.3.0/lib/libuuid.so.1: no version information available (required by /lib64/libSM.so.6)


In [5]:
%%bash
module load afni/20.1.11
cd /data/bioprotean/RAG2/AVG/MWT_avg/to_allen/overlap/high_res/
3dresample \
    -dxyz 0.015 0.015 0.015 \
    -input MKO_MWTavg_invjcb_avg_thrsh_1set_qw_masked_neg_ctrst_clust_binary_75um.nii \
    -prefix MKO_MWTavg_invjcb_avg_thrsh_1set_qw_masked_neg_ctrst_clust_binary_15um.nii

3dresample: /packages/7x/anaconda3/5.3.0/lib/libuuid.so.1: no version information available (required by /lib64/libSM.so.6)
